<a href="https://colab.research.google.com/github/vadim-privalov/Neiroset_Novosibirsk/blob/main/%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B5%D0%BC_%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE%D0%BF%D0%BE%D1%82%D0%BE%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Распознаем видеопотоки

# Recognizing video streams


В этом уроке мы научимся распознавать текст с визитной карточки на видео. Для этого мы воспользуемся уже полученными знаниями распознавания видеопотоков, применим FFT для для обнаружения и удаления размытых и некачественных кадров и выведем все.

In this lesson we will learn how to recognize text from a business card on a video. To do this we will use the knowledge we already have of video stream recognition, apply FFT to detect and remove blurry and low-quality frames and output everything.

Нам необходимо будет реализовать базовую вспомогательную функцию, которая позволит нам записать выходные данные нашего сценария распознавания видео на диск в виде отдельного выходного видео.

We will need to implement a basic auxiliary function that will allow us to write the output of our video recognition scenario to disk as a separate output video.

In [1]:
!wget http://dataudt.ru/datasets/cv/Lesson_17.OCR_video.zip
!unzip Lesson_17.OCR_video.zip
%cd /content/OCR_video


--2022-02-10 11:51:13--  http://dataudt.ru/datasets/cv/Lesson_17.OCR_video.zip
Resolving dataudt.ru (dataudt.ru)... 37.228.117.130
Connecting to dataudt.ru (dataudt.ru)|37.228.117.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24947046 (24M) [application/zip]
Saving to: ‘Lesson_17.OCR_video.zip’

Lesson_17.OCR_video 100%[===================>]  23.79M  38.3MB/s    in 0.6s    

2022-02-10 11:51:14 (38.3 MB/s) - ‘Lesson_17.OCR_video.zip’ saved [24947046/24947046]

Archive:  Lesson_17.OCR_video.zip
   creating: OCR_video/blur_detection/
   creating: OCR_video/blur_detection/__pycache__/
  inflating: OCR_video/blur_detection/__pycache__/blur_detector.cpython-36.pyc  
  inflating: OCR_video/blur_detection/blur_detector.py  
  inflating: OCR_video/helpers.py    
  inflating: OCR_video/ocr_video.py  
   creating: OCR_video/output/
  inflating: OCR_video/output/ocr_video_output.avi  
  inflating: OCR_video/rus.traineddata  
   creating: OCR_video/video/
  inflating

но сперва настроим работу tesseract 

But first let's set up the tesseract 





In [2]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [3]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

Если будем использовать русский язык, то переместим и этот файл 

If we use Russian, we will move this file as well 


In [4]:
!mv "/content/OCR_video/rus.traineddata" "/usr/share/tesseract-ocr/4.00/tessdata/"

In [5]:
# импортируем необходимые пакеты
# import the necessary packages
import numpy as np
from imutils.video import VideoStream
# для  перспективного преобразования (вид сверху) 
# for perspective transformation (top view)
from imutils.perspective import four_point_transform 
from pytesseract import Output
import pytesseract
import imutils
import time
import cv2

А теперь перейдем к самой функции:

Now let's move on to the function itself:

Функция `build` внутри класса `VideoOCROutputBuilder` принимает следующие параметры:

The `build` function inside the `VideoOCROutputBuilder` class takes the following parameters:

- `frame` - входной кадр из видео. the input frame from the video.
- `card` - Визитная карточка после применения преобразования перспективы сверху вниз и обнаружения текста на карте. The calling card after applying a top-down perspective transformation and detecting text on the card.
- `ocr` -  распознанный текст OCR. OCR recognized text .




In [6]:
class VideoOCROutputBuilder:
	def __init__(self, frame):
        # храним размеры входного кадра
        # store the input frame dimensions
		self.maxW = frame.shape[1]
		self.maxH = frame.shape[0]

	def build(self, frame, card=None, ocr=None):
        # получаем размеры входного кадра и инициализируем 
        # размеры изображения визитной карточки вместе
        #  с размерами изображения OCR
        # grab the input frame dimensions and  initialize the card
		# image dimensions along with the OCR image dimensions
		(frameH, frameW) = frame.shape[:2]
		(cardW, cardH) = (0, 0)
		(ocrW, ocrH) = (0, 0)

        # если изображение визитной карточки
        # не пустое, то возьмем его размеры
        # if the card image is not empty, grab its dimensions
		if card is not None:
			(cardH, cardW) = card.shape[:2]

        # аналогично, если изображение OCR не пустое, 
        # то возьмем его размеры
        # similarly, if the OCR image is not empty, grab its
		# dimensions
		if ocr is not None:
			(ocrH, ocrW) = ocr.shape[:2]

        # вычисляем пространственные размеры выходного кадра
        # compute the spatial dimensions of the output frame
		outputW = max([frameW, cardW, ocrW])
		outputH = frameH + cardH + ocrH

        # обновляем максимальный выходной размер,
        # найденный на данный момент
        # update the max output spatial dimensions found thus far
		self.maxW = max(self.maxW, outputW)
		self.maxH = max(self.maxH, outputH)

        # выделяем память для выходного изображения,
        # используя найденный выше максимальный выходной размер
        # allocate memory of the output image using our maximum
		# spatial dimensions
		output = np.zeros((self.maxH, self.maxW, 3), dtype="uint8")

        # устанавливаем рамку в выходном изображении
        # set the frame in the output image
		output[0:frameH, 0:frameW] = frame

        # если карточка была найдена, то 
        # добавляем ее к выходному изображению
        # if the card is not empty, add it to the output image
		if card is not None:
			output[frameH:frameH + cardH, 0:cardW] = card

        # если результат OCR не пуст, то 
        # добавляем его к выходному изображению
        # if the OCR result is not empty, add it to the output image
		if ocr is not None:
			output[
				frameH + cardH:frameH + cardH + ocrH,
				0:ocrW] = ocr

        # возвращаем полученное изображение 
        # return the output visualization image
		return output

Так же еще допишем функции, с которой мы уже знакомы 


Let's also add a function that we are already familiar with 


In [7]:
def detect_blur_fft(image, size=60, thresh=10):
    # получим размеры изображения и используем 
    # их для получения центра координат
    # grab the dimensions of the image and use the dimensions to
	# derive the center (x, y)-coordinates
	(h, w) = image.shape
	(cX, cY) = (int(w / 2.0), int(h / 2.0))

    # вычисляем FFT для нахождения частотного преобразования, 
    # используя встроенную функцию в numpy.
    # Затем мы смещаем нулевую частотную к центру,
    # где ее будет легче анализировать
    # compute the FFT to find the frequency transform, then shift
	# the zero frequency component (i.e., DC component located at
	# the top-left corner) to the center where it will be more
	# easy to analyze
	fft = np.fft.fft2(image)
	fftShift = np.fft.fftshift(fft)

    # обнуляем центр сдвига FFT (т.е. удаляем низкие частоты),
    # применяем обратный сдвиг так, а затем применяем
    # обратное FFT
    # zero-out the center of the FFT shift (i.e., remove low
	# frequencies), apply the inverse shift such that the DC
	# component once again becomes the top-left, and then apply
	# the inverse FFT
	fftShift[cY - size:cY + size, cX - size:cX + size] = 0
	fftShift = np.fft.ifftshift(fftShift)
	recon = np.fft.ifft2(fftShift)

    # вычисляем спектр магнитуд восстановленного изображения,
    # затем вычисляем их среднее значение 
    # compute the magnitude spectrum of the reconstructed image,
	# then compute the mean of the magnitude values
	magnitude = 20 * np.log(np.abs(recon))
	mean = np.mean(magnitude)

    # изображение будет считаться "размытым", если 
    # среднее значение величин меньше порогового значения
    # the image will be considered "blurry" if the mean value of the
	# magnitudes is less than the threshold value
	return (mean, mean <= thresh)

И еще одна 

And another one. 


In [8]:
# дополнительная функция для очистки текста
# additional function to clear text
def cleanup_text(text):
    # вырезаем не-ASCII текст, чтобы мы могли нарисовать его на изображении 
    # с помощью OpenCV
    # strip out non-ASCII text so we can draw the text on the image
	# using OpenCV
	return "".join([c if (ord(c) < 128)  or ((ord(c) > 1039) and (ord(c) < 1104))  else "" for c in text]).strip()

Переходим к основному блоку. Реализуем скрипт, который будет распознавать видео 

Let's move on to the main block. Let's implement a script that will recognize video 


In [9]:
# создадим отдельный словарь с аргументами 
# create a separate dictionary with arguments 

args = {
  "input" : "video/ex1.MOV", # путь к входному видео 
                                # input video path 
  "output": 'output/out.avi', # путь к выходному видео
                        # the way to the output video
  "min-conf": 50, #  минимальное значение достоверности для фильтрации слабого обнаружения текста
                    # minimum validity value to filter out weak text detection
}



In [10]:
# инициализируем наш констуктор вывода видео OCR, используемый 
# для простой визуализации вывода на экран
# initialize our video OCR output builder used to easily visualize
# output to our screen
outputBuilder = None

# инициализируем переменную, которая будет записывать 
# выходное видео  и переменные с размерами выходного кадра
# initialize our output video writer along with the dimensions of the
# output frame
writer = None
outputW = None
outputH = None

# берем видеофайл
# take a video file
print("[INFO] opening video file...")
vs = cv2.VideoCapture(args["input"])

[INFO] opening video file...


In [11]:
# перебираем кадры из видеопотока
# loop over frames from the video stream
while True:
    # захватываем следующий кадр и обрабатываем
    # grab the next frame and handle if we are reading from either
	# a webcam or a video file
	orig = vs.read()
	orig = orig[1]

    # если мы еще просматриваем видео, но не 
    # захватили ни одного кадра, значит,
    # мы достигли конца видео
    # if we are viewing a video and we did not grab a frame then we
	# have reached the end of the video
	if orig is None:
		break

    # изменяем размер рамки и вычисляем отношение
    # *новой* ширины к *старой* ширине
    # resize the frame and compute the ratio of the *new* width to
	# the *old* width
	frame = imutils.resize(orig, width=600)
	ratio = orig.shape[1] / float(frame.shape[1])

    # если наш outputBuilder для OCR видео = None, 
    # то инициализируем его
    # if our video OCR output builder is None, initialize it
	if outputBuilder is None:
		outputBuilder = VideoOCROutputBuilder(frame)

    # инициализируем место для распознанной визитной 
    # карточки и ее OCR
    # initialize our card and OCR output ROIs
	card = None
	ocr = None

    # преобразуем кадр в в серые оттенки
    # и определяем, считается ли кадр размытым или нет (на основе уже
    #  написанной нами функцией)
    # convert the frame to grayscale and detect if the frame is
	# considered blurry or not
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	(mean, blurry) = detect_blur_fft(gray, thresh=15)

    # выводим прямо поверх картинки, размыт кадр или нет
    # draw whether or not the frame is blurry
	color = (0, 0, 255) if blurry else (0, 255, 0)
	text = "Blurry ({:.4f})" if blurry else "Not Blurry ({:.4f})"
	text = text.format(mean)
	cv2.putText(frame, text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,
		0.7, color, 2)

    # продолжаем обрабатывать кадр для OCR, 
    # только если изображение *не* размыто
    # only continue to process the frame for OCR if the image is
	# *not* blurry
	if not blurry:
        # слегка размываем полутоновое изображение,
        # а затем выполняем определение краев
        # blur the grayscale image slightly and then perform edge
		# detection
		blurred = cv2.GaussianBlur(gray, (5, 5,), 0)
		edged = cv2.Canny(blurred, 75, 200)

        # находим контуры на карте граней и сортируем их по размеру 
        # в порядке убывания, сохраняя только самые крупные из них
        # find contours in the edge map and sort them by size in
		# descending order, keeping only the largest ones
		cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
			cv2.CHAIN_APPROX_SIMPLE)
		cnts = imutils.grab_contours(cnts)
		cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]

        # инициализируем контур, который соответствует
        # контуру визитной карточки
        # initialize a contour that corresponds to the business card
		# outline
		cardCnt = None

        # проходимся по контурам
        # loop over the contours
		for c in cnts:
            # аппроксимируем контур
            # approximate the contour
			peri = cv2.arcLength(c, True)
			approx = cv2.approxPolyDP(c, 0.02 * peri, True)

            # если наш приближенный контур имеет четыре точки,
            #  то можно считать, что мы нашли контур визитной карточки
            # if our approximated contour has four points, then we
			# can assume we have found the outline of the business
			# card
			if len(approx) == 4:
				cardCnt = approx
				break

        # убедились, что контур визитной карточки был найден
        # ensure that the business card contour was found
		if cardCnt is not None:
            # рисуем контур визитной карточки на рамке, чтобы 
            # убедиться, что она была обнаружена правильно
            # draw the outline of the business card on the frame so
			# we visually verify that the card was detected correctly
			cv2.drawContours(frame, [cardCnt], -1, (0, 255, 0), 3)

            # применим преобразование перспективы в четырех точках к 
            # *оригинальному* кадру, чтобы получить вид визитной карточки
            # сверху 
            # apply a four point perspective transform to the
			# *original* frame to obtain a top-down birds eye
			# view of the business card
			card = four_point_transform(orig,
				cardCnt.reshape(4, 2) * ratio)

            # выделяем память для нашей выходной визуализации OCR
            # allocate memory for our output OCR visualization
			ocr = np.zeros(card.shape, dtype="uint8")

            # меняем  канал  визитной карточки и распознаем ее
            # swap channel ordering for the business card and OCR it
			rgb = cv2.cvtColor(card, cv2.COLOR_BGR2RGB)
			results = pytesseract.image_to_data(rgb,
				output_type=Output.DICT, lang='eng')

            # перебираем каждую отдельную текстовую локализацию
            # loop over each of the individual text localizations
			for i in range(0, len(results["text"])):
                # извлекаем координаты ограничивающей 
                # рамки текстовой области из текущего результата
                # extract the bounding box coordinates of the text
				# region from the current result
				x = results["left"][i]
				y = results["top"][i]
				w = results["width"][i]
				h = results["height"][i]

                # извлекаем сам текст OCR вместе с
                # вероятностью в локализации текста
                # extract the OCR text itself along with the
				# confidence of the text localization
				text = results["text"][i]
				conf = int(results["conf"][i])

                # отфильтровываем текстовые локализации со слабым доверием
                # filter out weak confidence text localizations
				if conf > args["min-conf"]:
                    # обрабатываем текст, удаляя из него символы,
                    # не являющиеся символами ASCII
                    # process the text by stripping out non-ASCII
					# characters
					text = cleanup_text(text)

                    # если очищенный текст не пустой, 
                    # то рисуем ограничивающую рамку вокруг текста
                    # вместе с самим текстом
                    # if the cleaned up text is not empty, draw a
					# bounding box around the text along with the
					# text itself
					if len(text) > 0:
						cv2.rectangle(card, (x, y), (x + w, y + h),
							(0, 255, 0), 2)
						cv2.putText(ocr,  text , (x, y - 10),
							cv2.FONT_HERSHEY_SIMPLEX, 0.5,
							(0, 0, 255), 1)

    # создаем окончательную визуализацию выходного видео
    # build our final video OCR output visualization
	output = outputBuilder.build(frame, card, ocr)

    # проверяем, стоит ли записывать видео в новый файл 
    # check if the video writer is None *and* an output video file
	# path was supplied
	if args["output"] is not None and writer is None:
        # получаем размеры выходного кадра и 
        # инициализируем VideoWriter
        # grab the output frame dimensions and initialize our video
		# writer
		(outputH, outputW) = output.shape[:2]
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 27,
			(outputW, outputH), True)

    # если writer не пуст, то нужно записать выходную 
    # визуализацию видео OCR на диск
    # if the writer is not None, we need to write the output video
	# OCR visualization to disk
	if writer is not None:
        # принудительно изменяем размер визуализации
        # OCR видео в соответствии с размерами выходного видео
        # force resize the video OCR visualization to match the
		# dimensions of the output video
		outputFrame = cv2.resize(output, (outputW, outputH))
		writer.write(outputFrame)


vs.release()

Видео воспроизводить будем по следующему принципу:

We will play the video in the following way:

- получаем видео в формате avi. get video in avi format.
- конвертирую его в mp4. convert it to mp4 .
- воспроизводим с помощью модуля kora. play it using the kora module.



In [12]:
!pip install -U kora

     |████████████████████████████████| 57 kB 4.3 MB/s 
     |████████████████████████████████| 56 kB 2.2 MB/s 


In [13]:
!ls

blur_detection	helpers.py  ocr_video.py  output  video  video_ocr


In [14]:
!ffmpeg -i output/out.avi output/out.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

Для корректной работы нам необходимо подключить модуль kora, который просит подключиться к гугл диску. Для этого мы при запуске блока кода ниже должны перейти по появившейся ссылки и после соглашения скопировать ссылку и вставить сюда. 

In [15]:
from kora.drive import upload_public
url = upload_public('output/out.mp4')

from IPython.display import HTML
HTML(f"""<video src={url} width=640 controls/>""")